### SQL Databases

In [6]:
## create sample SQLite Database
import sqlite3
import os

os.makedirs("data/databases",exist_ok=True)


In [7]:
## create sample database
conn=sqlite3.connect('data/databases/company.db')
cursor=conn.cursor()

In [8]:
# Create tables
cursor.execute('''CREATE TABLE IF NOT EXISTS employees
                 (id INTEGER PRIMARY KEY, name TEXT, role TEXT, department TEXT, salary REAL)''')

In [9]:
cursor.execute('''CREATE TABLE IF NOT EXISTS projects
                 (id INTEGER PRIMARY KEY, name TEXT, status TEXT, budget REAL, lead_id INTEGER)''')

In [10]:
# Insert sample data
employees = [
    (1, 'John Doe', 'Senior Developer', 'Engineering', 95000),
    (2, 'Jane Smith', 'Data Scientist', 'Analytics', 105000),
    (3, 'Mike Johnson', 'Product Manager', 'Product', 110000),
    (4, 'Sarah Williams', 'DevOps Engineer', 'Engineering', 98000)
]

projects = [
    (1, 'RAG Implementation', 'Active', 150000, 1),
    (2, 'Data Pipeline', 'Completed', 80000, 2),
    (3, 'Customer Portal', 'Planning', 200000, 3),
    (4, 'ML Platform', 'Active', 250000, 2)
]


In [11]:
cursor.executemany('INSERT OR REPLACE INTO employees VALUES (?,?,?,?,?)', employees)
cursor.executemany('INSERT OR REPLACE INTO projects VALUES (?,?,?,?,?)', projects)

In [12]:
cursor.execute("Select * from employees")

In [13]:
conn.commit()
conn.close()


## Database Content Extraction

In [20]:
from langchain_community.utilities import SQLDatabase
from langchain_community.document_loaders import SQLDatabaseLoader


In [22]:
## MEthod1 SQLDATABASE Utility
db = SQLDatabase.from_uri("sqlite:///data/databases/company.db")

## get Database info
print(f"Tables: {db.get_usable_table_names()}")
print(f"\nTable DDL:")
print(db.get_table_info())

Tables: ['employees', 'projects']

Table DDL:

CREATE TABLE employees (
	id INTEGER, 
	name TEXT, 
	role TEXT, 
	department TEXT, 
	salary REAL, 
	PRIMARY KEY (id)
)

/*
3 rows from employees table:
id	name	role	department	salary
1	John Doe	Senior Developer	Engineering	95000.0
2	Jane Smith	Data Scientist	Analytics	105000.0
3	Mike Johnson	Product Manager	Product	110000.0
*/


CREATE TABLE projects (
	id INTEGER, 
	name TEXT, 
	status TEXT, 
	budget REAL, 
	lead_id INTEGER, 
	PRIMARY KEY (id)
)

/*
3 rows from projects table:
id	name	status	budget	lead_id
1	RAG Implementation	Active	150000.0	1
2	Data Pipeline	Completed	80000.0	2
3	Customer Portal	Planning	200000.0	3
*/


In [25]:
# We import List from typing so we can clearly declare
# that this function will return a list of objects.
# This is not required for Python to run, but it helps
# humans (and tools) understand the function output.
from typing import List

# We import the Document class from LangChain.
# A Document is the basic unit used in RAG pipelines.
# Each Document contains:
#   1. page_content -> the text an LLM will read
#   2. metadata     -> extra information for filtering and traceability
from langchain_core.documents import Document

# This print statement is only for visual clarity in a notebook.
# It helps the user see that a new processing section has started.
# It has no effect on the program logic.
print("\n2️⃣ Custom SQL Processing")


def sql_to_documents(db_path: str) -> List[Document]:
    """
    PURPOSE OF THIS FUNCTION:

    This function converts a structured SQL database into
    unstructured text Documents that an LLM can understand.

    WHY THIS IS NEEDED:
    - LLMs cannot directly query SQL databases.
    - Vector databases work with text, not tables.
    - We therefore convert tables + relationships into readable text
      while preserving meaning and context.

    INPUT:
    - db_path: Path to a SQLite database file

    OUTPUT:
    - A list of LangChain Document objects ready for embedding and RAG
    """

    # -----------------------------
    # STEP 1: Connect to the database
    # -----------------------------

    # This opens a connection to the SQLite database file.
    # SQLite databases are stored as files, so this gives Python
    # direct access to the data inside the file.
    conn = sqlite3.connect(db_path)

    # The cursor is the object that actually runs SQL commands.
    # Think of it as a "pointer" that can send queries to the database
    # and receive results back.
    cursor = conn.cursor()

    # This list will store every Document we create.
    # At the end of the function, we return this list.
    documents = []

    # ==========================================================
    # STRATEGY 1: CREATE ONE DOCUMENT PER TABLE
    # ==========================================================
    #
    # WHY THIS STRATEGY?
    # - Tables represent logical data domains (employees, orders, etc.)
    # - Creating one document per table preserves this structure
    # - It allows the LLM to reason about tables independently
    #

    # -----------------------------------
    # STEP 2: Find all tables in database
    # -----------------------------------

    # SQLite stores metadata about the database in a special table
    # called sqlite_master.
    # This query retrieves the names of all tables in the database.
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

    # fetchall() returns all rows from the query.
    # Each row is a tuple, even if it contains only one value.
    tables = cursor.fetchall()

    # Loop through each table found in the database.
    for table in tables:

        # Extract the table name from the tuple.
        # Example: ('employees',) -> 'employees'
        table_name = table[0]

        # -----------------------------------------
        # STEP 3: Extract table schema (column info)
        # -----------------------------------------

        # PRAGMA table_info is a SQLite command that returns
        # detailed information about each column:
        # - column name
        # - data type
        # - constraints
        #
        # WHY THIS IS IMPORTANT:
        # - LLMs need schema context to interpret data correctly
        # - Column names act as semantic labels
        cursor.execute(f"PRAGMA table_info({table_name});")

        # Fetch all column metadata
        columns = cursor.fetchall()

        # Extract only column names.
        # Column name is stored at index 1 of each result row.
        column_names = [col[1] for col in columns]

        # --------------------------------
        # STEP 4: Retrieve table data
        # --------------------------------

        # Retrieve all rows from the table.
        # We do this so we can:
        # - count records
        # - sample representative data
        cursor.execute(f"SELECT * FROM {table_name}")

        # Fetch all rows from the table.
        rows = cursor.fetchall()

        # -----------------------------------------
        # STEP 5: Build readable table description
        # -----------------------------------------
        #
        # WHY WE DO THIS:
        # - LLMs understand text, not raw SQL rows
        # - We convert structure into natural language
        # - We provide high-level summaries + examples
        #

        # Start the document content with the table name.
        table_content = f"Table: {table_name}\n"

        # Add column names as a comma-separated list.
        table_content += f"Columns: {', '.join(column_names)}\n"

        # Add record count so the LLM understands scale.
        table_content += f"Total Records: {len(rows)}\n\n"

        # Add a section header for example records.
        table_content += "Sample Records:\n"

        # Only include a small sample of rows.
        # WHY:
        # - Avoid very large documents
        # - Provide representative examples
        for row in rows[:5]:

            # Convert each row from a tuple into a dictionary.
            # This preserves column-value relationships clearly.
            record = dict(zip(column_names, row))

            # Append the readable record to the document text.
            table_content += f"{record}\n"

        # -----------------------------------------
        # STEP 6: Wrap table content as a Document
        # -----------------------------------------

        # Create a LangChain Document object.
        # This is the unit that will be embedded and indexed.
        doc = Document(
            page_content=table_content,
            metadata={
                # Helps trace answers back to the source database
                'source': db_path,

                # Allows filtering by table during retrieval
                'table_name': table_name,

                # Useful for debugging and ranking
                'num_records': len(rows),

                # Distinguishes this from relationship documents
                'data_type': 'sql_table'
            }
        )

        # Add the table document to our document list.
        documents.append(doc)

    # ==========================================================
    # STRATEGY 2: CREATE RELATIONSHIP DOCUMENTS (JOINS)
    # ==========================================================
    #
    # WHY THIS STRATEGY?
    # - SQL databases store meaning in relationships
    # - Joins capture business logic (who works on what)
    # - LLMs reason much better with explicit relationships
    #

    # This SQL query joins two tables to create semantic meaning.
    cursor.execute("""
        SELECT e.name, e.role, p.name AS project_name, p.status
        FROM employees e
        JOIN projects p ON e.id = p.lead_id
    """)

    # Fetch all joined rows.
    relationships = cursor.fetchall()

    # Start building natural language description.
    rel_content = "Employee-Project Relationships:\n\n"

    # Convert each joined row into a readable sentence.
    for rel in relationships:
        rel_content += (
            f"{rel[0]} ({rel[1]}) leads {rel[2]} "
            f"- Status: {rel[3]}\n"
        )

    # Wrap relationship text into a Document.
    rel_doc = Document(
        page_content=rel_content,
        metadata={
            'source': db_path,
            'data_type': 'sql_relationships',
            'query': 'employee_project_join'
        }
    )

    # Add relationship document to the document list.
    documents.append(rel_doc)

    # -----------------------------
    # STEP 7: Clean up and return
    # -----------------------------

    # Always close the database connection.
    # This prevents file locks and memory leaks.
    conn.close()

    # Return all generated documents.
    # These documents are now ready for:
    # - chunking
    # - embedding
    # - vector storage
    # - RAG querying
    return documents



2️⃣ Custom SQL Processing


In [26]:
sql_to_documents("data/databases/company.db")

[Document(metadata={'source': 'data/databases/company.db', 'table_name': 'employees', 'num_records': 4, 'data_type': 'sql_table'}, page_content="Table: employees\nColumns: id, name, role, department, salary\nTotal Records: 4\n\nSample Records:\n{'id': 1, 'name': 'John Doe', 'role': 'Senior Developer', 'department': 'Engineering', 'salary': 95000.0}\n{'id': 2, 'name': 'Jane Smith', 'role': 'Data Scientist', 'department': 'Analytics', 'salary': 105000.0}\n{'id': 3, 'name': 'Mike Johnson', 'role': 'Product Manager', 'department': 'Product', 'salary': 110000.0}\n{'id': 4, 'name': 'Sarah Williams', 'role': 'DevOps Engineer', 'department': 'Engineering', 'salary': 98000.0}\n"),
 Document(metadata={'source': 'data/databases/company.db', 'table_name': 'projects', 'num_records': 4, 'data_type': 'sql_table'}, page_content="Table: projects\nColumns: id, name, status, budget, lead_id\nTotal Records: 4\n\nSample Records:\n{'id': 1, 'name': 'RAG Implementation', 'status': 'Active', 'budget': 150000.